In [18]:
import seaborn as sns
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy.signal import square, stft

from scipy.signal.windows import gaussian


In [126]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [4]:
from DataLoader import *

In [176]:
data = DataLoader().load_ACC_data(subject='s2',event='evening')

In [178]:
data.drop(columns='UNIX_time',inplace=True)

In [204]:
data[1:2]

,X_acceleration,Y_acceleration,Z_acceleration
1,-35.0,-55.0,-7.0


In [208]:
data = data.to_numpy().flatten()

In [209]:
len(data)

172800

In [182]:
len(data)/3 

19200.0

In [344]:
datas_ = []
for i in range(1,24):
    subject_ = f's{i}'
    data = DataLoader().load_ACC_data(subject=subject_,event='evening')
    data.drop(columns='UNIX_time',inplace=True)
    data = data.to_numpy().flatten()
    datas_.append(data)

In [343]:
datas = []
for i in range(1,24):
    subject_ = f's{i}'
    data = DataLoader().load_ACC_data(subject=subject_,event='morning')
    data.drop(columns='UNIX_time',inplace=True)
    data = data.to_numpy().flatten()
    datas.append(data)

In [345]:
datas_

[array([-63., -15.,  25., ..., -66.,  -4.,  -3.]),
 array([-62., -17.,  21., ..., -42., -38.,  24.]),
 array([-58., -12.,  30., ...,  -3.,  35., -68.]),
 array([-48., -45.,   4., ..., -57., -34.,   1.]),
 array([-60., -16., -22., ..., -12.,  39.,  50.]),
 array([-60.,  -2.,  37., ..., -58.,  10.,  27.]),
 array([-51.,  30.,  28., ..., -24.,  54.,   5.]),
 array([-42.,  -8., -46., ..., -45.,  -4.,  21.]),
 array([-60.,   9.,  26., ..., -46., -44.,  19.]),
 array([-66.,   2.,   4., ..., -63., -12.,  11.]),
 array([-47.,  -7.,  46., ..., -50., -10., -44.]),
 array([-13.,   2.,  62., ...,  -9.,  36.,  51.]),
 array([-64., -11.,  15., ..., -66.,  -9.,   1.]),
 array([-65., -13.,   4., ..., -15., -25.,  59.]),
 array([-23., -12.,  60., ..., -41., -50.,  11.]),
 array([-15.,   4.,  63., ..., -26., -58.,  17.]),
 array([ -2., -27.,  78., ..., -27.,  -5.,  59.]),
 array([-63., -10., -18., ..., -16.,   4.,  63.]),
 array([-57.,  -7.,  31., ...,  -9.,  -8., -62.]),
 array([-37.,  11.,  53., ..., 

In [347]:
array = np.concatenate((datas, datas_), axis=0)

In [348]:
array.shape

(46, 172800)

In [230]:
datas

[array([-62.,  18.,  -2., ..., -58.,  19.,  33.]),
 array([-28., -20.,  56., ..., -43., -49.,   8.]),
 array([-58.,  -6., -18., ..., -53.,   2.,  43.]),
 array([-51.,  39.,   9., ..., -37.,  -8., -52.]),
 array([-47., -16.,  45., ..., -31.,  54.,  15.]),
 array([-30., -17.,  57., ..., -38.,   1.,  54.]),
 array([-28.,  57.,  -6., ..., -61.,  17., -16.]),
 array([-50.,  16.,  43., ..., -51.,  15.,  36.]),
 array([-57., -30.,  17., ..., -57., -31.,  16.]),
 array([-11.,  22.,  59., ..., -62.,  -7., -22.]),
 array([ -2.,  60.,  21., ...,  10., -52.,  34.]),
 array([-14., -24.,  59., ..., -28.,   6., -56.]),
 array([-55.,   0.,  38., ..., -34.,  -4.,  56.]),
 array([-50., -12.,  43., ..., -46., -47.,   3.]),
 array([-30.,   0.,  58., ..., -66.,  -1.,   6.]),
 array([-16.,  -1.,  62., ..., -25., -54.,  -4.]),
 array([-33.,   0.,  57., ..., -20.,   1.,  62.]),
 array([-57.,  -2.,  34., ..., -31.,   0.,  58.]),
 array([-59.,  11.,  28., ..., -41.,  23.,  44.]),
 array([  0.,  18.,  61., ..., 

In [350]:
datas = pd.DataFrame(array)

In [232]:
datas.shape

(23, 172800)

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler 

In [41]:
from sklearn.pipeline import make_pipeline

In [43]:
num_transformer=make_pipeline(StandardScaler())

In [49]:
num_transformer.fit(acc_s1m)

Pipeline(steps=[('standardscaler', StandardScaler())])

In [51]:
acc_s1m=num_transformer.transform(acc_s1m)

In [54]:
pd.DataFrame(acc_s1m)

,0,1,2
0,-0.743233,-0.771796,0.160711
1,-0.874754,-0.629046,0.004586
2,-0.874754,-0.533879,-0.073476
3,-0.874754,-0.438712,-0.112508
4,-0.808993,-0.438712,-0.190570
...,...,...,...
57595,-0.940514,-0.248379,-0.854102
57596,-0.940514,-0.248379,-0.854102
57597,-0.940514,-0.248379,-0.893133
57598,-0.940514,-0.248379,-0.932164


In [233]:
def normalization(subject):
    num_transformer=make_pipeline(StandardScaler())
    num_transformer.fit(subject)
    return pd.DataFrame(num_transformer.transform(subject),columns=subject.columns)

In [352]:
datas.shape

(46, 172800)

In [353]:
df=normalization(datas)

In [355]:
X = df

In [220]:
info = pd.read_excel('data/general_info.xlsx')

In [221]:
info

,subjects,age,gender,education,employment,marital status,smoking,sleep status,salary,chronic disease,drug status,gym status,duration of employment,last sleep time,morning-mental fatigue state,evening-mental fatigue state
0,S1,27,male,associate degree,full,single,no,no,10000-15000,no,no,yes,9,5,11,14
1,S2,33,female,associate degree,full,married,no,yes,10000-15000,no,no,no,9,8,18,14
2,S3,23,male,high school,full,single,yes,no,5000-10000,no,no,no,9,4,0,14
3,S4,25,male,associate degree,full,single,no,no,5000-10000,no,no,yes,9,7,12,12
4,S5,29,male,high school,full,single,no,yes,5000-10000,no,no,yes,9,8,10,13
5,S6,28,male,associate degree,full,single,no,yes,5000-10000,no,no,no,9,7,11,11
6,S7,27,male,high school,full,married,no,yes,10000-15000,no,no,yes,9,7,5,1
7,S8,19,male,high school,full,single,no,no,5000-10000,no,no,no,9,6,11,11
8,S9,31,female,graduate degree,full,married,no,no,15000-20000,no,no,no,9,7,2,13
9,S10,25,male,high school,full,single,yes,yes,10000-20000,no,no,yes,8,7,5,11


In [357]:
sleepy_df

,fatigue,is_tired
0,11,0
1,18,1
2,0,0
3,12,1
4,10,0
5,11,0
6,5,0
7,11,0
8,2,0
9,5,0


In [365]:
y = sleepy_df.is_tired

In [366]:
y

0     0
1     1
2     0
3     1
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    1
13    0
14    0
15    1
16    1
17    1
18    1
19    0
20    0
21    0
22    0
23    1
24    1
25    1
26    1
27    1
28    0
29    0
30    0
31    1
32    0
33    0
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    0
42    1
43    1
44    1
45    1
Name: is_tired, dtype: int64

In [360]:
from sklearn.neighbors import KNeighborsClassifier

In [367]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 42)

In [362]:
knn_model = KNeighborsClassifier()

In [369]:
knn_model = knn_model.fit(X_train,y_train)

In [370]:
cv_results = cross_validate(knn_model, X_train,y_train,cv=5,scoring='accuracy')

In [371]:
cv_results

{'fit_time': array([1.00537109, 0.9321301 , 0.94686413, 0.9549849 , 0.97174716]),
 'score_time': array([0.9557848 , 0.90182304, 0.91398287, 0.95510602, 0.91778493]),
 'test_score': array([0.85714286, 0.57142857, 0.66666667, 0.5       , 0.83333333])}

In [372]:
base_knn_score = cv_results['test_score'].mean()
base_knn_score

0.6857142857142857

In [283]:
from sklearn.model_selection import cross_val_score

In [319]:
from sklearn.model_selection import learning_curve

In [279]:
from sklearn.model_selection import cross_validate

In [375]:
from sklearn.metrics import accuracy_score

In [373]:
y_pred = knn_model.predict(X_test)

In [376]:
accuracy_score(y_test,y_pred)

0.5714285714285714

In [377]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [383]:
def test_classifiers(X_train, y_train, X_test, y_test):
    models = {
        'MLP Classifier': MLPClassifier(),  ## --> This is your baseline = chance
        'Decision Tree': DecisionTreeClassifier(),
        'Logistic Regression': LogisticRegression(),
        'Random Forest': RandomForestClassifier(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'K-Nearest Neighbors': KNeighborsClassifier()
        
    }

    results = {}

    for name, model in models.items():
        pipe = make_pipeline(model)
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        
        report = classification_report(y_test, y_pred)
        
        results[name] = {
            'accuracy': accuracy,
            'report': report
        }
        
        print(f"{name}:\n")
        print(f"Accuracy: {accuracy}\n")
        print(f"Classification Report:\n{report}\n")
        
    return results

In [384]:
test_classifiers(X_train, y_train, X_test, y_test)

MLP Classifier:

Accuracy: 0.6428571428571429

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.71      0.62      0.67         8

    accuracy                           0.64        14
   macro avg       0.64      0.65      0.64        14
weighted avg       0.65      0.64      0.64        14


Decision Tree:

Accuracy: 0.5714285714285714

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.67      0.57         6
           1       0.67      0.50      0.57         8

    accuracy                           0.57        14
   macro avg       0.58      0.58      0.57        14
weighted avg       0.60      0.57      0.57        14


Logistic Regression:

Accuracy: 0.6428571428571429

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.

{'MLP Classifier': {'accuracy': 0.6428571428571429,
  'report': '              precision    recall  f1-score   support\n\n           0       0.57      0.67      0.62         6\n           1       0.71      0.62      0.67         8\n\n    accuracy                           0.64        14\n   macro avg       0.64      0.65      0.64        14\nweighted avg       0.65      0.64      0.64        14\n'},
 'Decision Tree': {'accuracy': 0.5714285714285714,
  'report': '              precision    recall  f1-score   support\n\n           0       0.50      0.67      0.57         6\n           1       0.67      0.50      0.57         8\n\n    accuracy                           0.57        14\n   macro avg       0.58      0.58      0.57        14\nweighted avg       0.60      0.57      0.57        14\n'},
 'Logistic Regression': {'accuracy': 0.6428571428571429,
  'report': '              precision    recall  f1-score   support\n\n           0       0.57      0.67      0.62         6\n           1 

In [414]:
from sklearn.model_selection import GridSearchCV
from scipy import stats

In [411]:
model = GradientBoostingClassifier()

In [ ]:
batch_size='auto',alpha=0.0001,learning_rate='constant',early_stopping=True,

In [412]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

In [415]:
grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1, 
                           verbose=2
             )

In [418]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


KeyboardInterrupt: 

In [417]:
search.best_estimator_


MLPClassifier(alpha=0.1)

In [402]:
model = MLPClassifier(alpha=0.001)

In [403]:
model.fit(X_train, y_train)

MLPClassifier(alpha=0.001)

In [404]:
accuracy = accuracy_score(y_test, y_pred)

In [405]:
report = classification_report(y_test, y_pred)

In [408]:
print(f"Accuracy: {accuracy}\n")
print(f"Classification Report:\n{report}\n")

Accuracy: 0.5714285714285714

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.17      0.25         6
           1       0.58      0.88      0.70         8

    accuracy                           0.57        14
   macro avg       0.54      0.52      0.47        14
weighted avg       0.55      0.57      0.51        14


